3. Exploratory Data Analysis  
3.1 Imports & loading Data  
3.2 Explore the data  
   * a. Features scanning & evaluation  
   * b. Visualization  
   * c. PCA tranformation if needed
* 3.3 Conclusion & summary

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy

In [121]:
mass_shooting = pd.read_csv('../data/Clean_data/MassShooting2009_2019_final.csv',index_col='date')

In [122]:
mass_shooting.info()

<class 'pandas.core.frame.DataFrame'>
Index: 244 entries, 2009-03-10 to 2018-11-19
Data columns (total 19 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   fatalities                      244 non-null    int64  
 1   injured                         244 non-null    int64  
 2   total_victims                   244 non-null    int64  
 3   policeman_killed                244 non-null    int64  
 4   age                             244 non-null    int64  
 5   employeed(Y/N)                  244 non-null    int64  
 6   employed_at                     244 non-null    object 
 7   mental_health_issues            244 non-null    object 
 8   gender                          244 non-null    object 
 9   year                            244 non-null    int64  
 10  state                           244 non-null    object 
 11  total_licensees                 244 non-null    object 
 12  licensed_business_entitie

In [123]:
mass_shooting['population'] = mass_shooting.population.apply(lambda x: np.int(x.replace(',','')) if type(x) == str else x)
mass_shooting['total_gun_deaths'] = mass_shooting['total_gun_deaths'].apply(lambda x: np.int(x.replace(',','')) if type(x) == str else x)
mass_shooting['total_children_teen_gun_deaths'] = mass_shooting['total_children_teen_gun_deaths'].apply(lambda x: np.int(x.replace(',','')) if type(x) == str else x)
mass_shooting['licensed_business_entities'] = mass_shooting['licensed_business_entities'].apply(lambda x: np.int(x.replace(',','')) if type(x) == str else x)

In [124]:
mass_shooting.head(2)

,fatalities,injured,total_victims,policeman_killed,age,employeed(Y/N),employed_at,mental_health_issues,gender,year,state,total_licensees,licensed_business_entities,population,total_gun_deaths,total_children_teen_gun_deaths,state_population,state_labor_force,unemployment_rate
date,,,,,,,,,,,,,,,,,,,
2009-03-10,10,6,16,0,28,-999,-999,No,Male,2009,Alabama,"115,395",60349,307006550,31347,2811,3621410.0,2162999.0,11.0
2009-03-29,8,2,10,0,45,-999,-999,Yes,Male,2009,North Carolina,"115,395",60349,307006550,31347,2811,7117828.0,4570789.0,10.6


In [125]:
sectors = {'retail': ['Weis grocery','Advance Granite Store'], 'it': ['Yahoo','Virginia TV'],
  'military': ['Fort Hood','Army veteran','former Marine','U.S Army','Military','Navy Yard','former military'],
  'manufacturing': ['manufacturing company','manufacturer Fiamma Inc.','factory'],
  'education': ['University of Alabama'], 'other': '-999',
  'hospitality': ['county health department','some nursing home'],
  'transportation': ['uber','FedEx']}
  
def get_sector(comp):
  for key, val in sectors.items():
    if comp in val:
      comp = key            
  return comp

mass_shooting['sectors'] = mass_shooting['employed_at'].apply(lambda x: get_sector(x))    

In [126]:
null_val = {-999.000000: 0}
gender_code = {'-999': 0,'Male': 1,'Female': 2,'Male and Female':3}
health_code = {'-999': 0,'Yes': 1,'No': 2}
sector_code = {'other':0,'it':1,'military':2,'hospitality':3,'retail':4,'manufacturing':5,
  'education':6,'transportation':7}

mass_shooting['policeman_killed'].replace(null_val, inplace=True)
mass_shooting['age'].replace(null_val, inplace=True)
mass_shooting['employeed(Y/N)'].replace(null_val, inplace=True)
mass_shooting.gender.replace(gender_code,inplace=True)
mass_shooting['mental_health_issues'].replace(health_code,inplace=True)
mass_shooting.sectors.replace(sector_code,inplace=True)

In [127]:
mass_shooting.describe()

,fatalities,injured,total_victims,policeman_killed,age,employeed(Y/N),mental_health_issues,gender,year,licensed_business_entities,population,total_gun_deaths,total_children_teen_gun_deaths,state_population,state_labor_force,unemployment_rate,sectors
count,244.000000,244.000000,244.000000,244.000000,244.000000,244.000000,244.000000,244.000000,244.000000,244.000000,2.440000e+02,244.000000,244.000000,2.440000e+02,2.440000e+02,244.000000,244.000000
mean,3.930328,6.245902,10.176230,0.118852,14.532787,0.090164,0.704918,0.946721,2014.819672,77285.282787,3.203861e+08,36521.954918,2930.758197,1.012367e+07,6.310475e+06,5.993852,0.299180
std,5.846393,34.181267,38.332712,0.564933,18.843599,0.287005,0.813351,0.427007,1.979385,5080.585517,4.408755e+06,2472.757342,280.727587,8.174163e+06,5.099195e+06,1.894937,1.083281
min,0.000000,0.000000,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2009.000000,60349.000000,3.070066e+08,31347.000000,2465.000000,5.380540e+05,3.741260e+05,2.800000,0.000000
25%,1.000000,1.000000,4.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2014.000000,77815.000000,3.189074e+08,33636.000000,2711.000000,4.727283e+06,3.075017e+06,4.700000,0.000000
50%,3.000000,3.000000,5.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2015.000000,79188.000000,3.208966e+08,36252.000000,2824.000000,7.758751e+06,4.769472e+06,5.500000,0.000000
75%,5.000000,4.000000,8.000000,0.000000,29.000000,0.000000,1.000000,1.000000,2016.000000,80119.000000,3.231275e+08,38658.000000,3155.000000,1.564546e+07,9.553634e+06,6.600000,0.000000
max,58.000000,527.000000,585.000000,5.000000,70.000000,1.000000,2.000000,3.000000,2018.000000,80493.000000,3.266875e+08,39773.000000,3443.000000,3.098267e+07,1.928082e+07,13.000000,7.000000


In [128]:
mass_shooting.info()

<class 'pandas.core.frame.DataFrame'>
Index: 244 entries, 2009-03-10 to 2018-11-19
Data columns (total 20 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   fatalities                      244 non-null    int64  
 1   injured                         244 non-null    int64  
 2   total_victims                   244 non-null    int64  
 3   policeman_killed                244 non-null    int64  
 4   age                             244 non-null    int64  
 5   employeed(Y/N)                  244 non-null    int64  
 6   employed_at                     244 non-null    object 
 7   mental_health_issues            244 non-null    int64  
 8   gender                          244 non-null    int64  
 9   year                            244 non-null    int64  
 10  state                           244 non-null    object 
 11  total_licensees                 244 non-null    object 
 12  licensed_business_entitie

## Clustering states by its population, unemployment rate, gun possession, education